In [22]:
import pandas_datareader.data as web
import datetime
import pandas as pd
from functools import reduce
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Start and ending times for dataset
startTrain = datetime.datetime(2009, 1, 1)
endTrain = datetime.datetime(2017, 1, 1)

# testing dataset
startTest = datetime.datetime(2017, 1, 2)
endTest = datetime.datetime(2019, 1, 1)

# List of stocks that are in the portfolio
# TODO: Let the user input a stock and create this array based on the inputs
# !: Removed gs-pj and fb as their data is lacking earlier years
stocks = ["AAPL", "CAH", "CMCSA", "DISH", "GOOG", "HSY", "JNJ", "JPM", "K", "MA", "NFLX", "UL", "WBA"]


In [24]:
def get_stock(ticker, start=startTrain, end=endTrain):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    # print(data)
    data[f'{ticker}'] = data["Close"]#(data["Close"] - data["Open"])/data["Open"]
    data = data[[f'{ticker}']] 
    # print(data.head())
    return data 

pfizer = get_stock("AAPL")
jnj = get_stock("NFLX")

In [25]:
def combine_stocks(tickers, testing=False):
    data_frames = []
    for i in tickers:
        if (testing):
            data_frames.append(get_stock(i, startTest, endTest))
        else:
            data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), data_frames)

    # print(df_merged.head())
    return df_merged

In [26]:
import yfinance as yf
def get_historical_data(ticker, start_date, end_date):
    # Pull Historical Data
    data = yf.download(ticker, start=start_date, end=end_date)
    # Calculate Daily Returns
    data['Daily Return'] = data['Adj Close'].pct_change()   
    return data.dropna()

In [27]:
def std_dev(data):
    # Get number of observations
    n = len(data)
    # Calculate mean
    mean = sum(data) / n
    # Calculate deviations from the mean
    deviations = sum([(x - mean)**2 for x in data])
    # Calculate Variance & Standard Deviation
    variance = deviations / (n - 1)
    s = variance**(1/2)
    return s

In [28]:
# Sharpe Ratio From Scratch
def sharpe_ratio(data, risk_free_rate=0.02): # risk_free_rate refers to 10 year treasury bond rate(2% is used)
    # Calculate Average Daily Return
    mean_daily_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = std_dev(data)
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate/252) / s #TODO: check if risk_free_rate should be divided by 252 trading days
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
    
    return sharpe_ratio

In [29]:
def get_stock_return(ticker, start=startTrain, end=endTrain):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    print(data)
    data[f'{ticker}'] = data["Close"]#(data["Close"] - data["Open"])/data["Open"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 

In [30]:
aapl = get_historical_data("AAPL", start_date="2009-01-01", end_date="2019-01-01")
aaplSharpe = sharpe_ratio(aapl['Daily Return'])
aapl['AAPL'] = aapl['Daily Return']
aapl.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Daily Return'], inplace=True)
# aapl

[*********************100%***********************]  1 of 1 completed


/var/folders/kq/16k2vljd2wn9ghsrfv91f0sw0000gn/T/ipykernel_37117/1330891303.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['AAPL'] = aapl['Daily Return']
/Users/royleongyewhwee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [50]:
portfolio_daily_changes = aapl
# data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Daily Return'], inplace=True)
for i in stocks:
    data = get_historical_data(i, start_date="2009-01-01", end_date="2019-01-01") # train + test time frame
    portfolio_daily_changes[i] = data['Daily Return']

[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
CAH
[*********************100%***********************]  1 of 1 completed
CMCSA
[*********************100%***********************]  1 of 1 completed
DISH
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
HSY
[*********************100%***********************]  1 of 1 completed
JNJ
[*********************100%***********************]  1 of 1 completed
JPM
[*********************100%***********************]  1 of 1 completed
K
[*********************100%***********************]  1 of 1 completed
MA
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
UL
[*********************100%***********************]  1 of 1 completed
WBA


In [ ]:
# str(portfolio_daily_changes.index[0].date())
# for i in portfolio_daily_changes.columns:
#     print(i)

portfolio = combine_stocks(stocks) #?: By default it's the training time frame which is from start 2009 to end 2016

# training dataset is used to create the portfolio model
mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# portfolio


In [47]:
# PORTFOLIO BASED ON TRAIN DATA
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights_best_sharpe = ef.clean_weights()
# print(dict(cleaned_weights_best_sharpe))
print("\nMax Sharpe (Best reward/risk ratio)")
ef.portfolio_performance(verbose=True)

print("\nPortfolio percentages for BEST SHARPE")
# adds up to roughly 100%. Might be slightly off due to the decimals
print(cleaned_weights_best_sharpe.items())
for key in cleaned_weights_best_sharpe:
    print(key, str(round(cleaned_weights_best_sharpe[key] * 100, 2)) + '%')


Max Sharpe (Best reward/risk ratio)
Expected annual return: 31.2%
Annual volatility: 20.7%
Sharpe Ratio: 1.41

Portfolio percentages for BEST SHARPE
odict_items([('AAPL', 0.41709), ('CAH', 0.0), ('CMCSA', 0.00698), ('DISH', 0.01731), ('GOOG', 0.00761), ('HSY', 0.19678), ('JNJ', 0.0), ('JPM', 0.0), ('K', 0.0), ('MA', 0.17018), ('NFLX', 0.17273), ('UL', 0.0), ('WBA', 0.01132)])
AAPL 41.71%
CAH 0.0%
CMCSA 0.7%
DISH 1.73%
GOOG 0.76%
HSY 19.68%
JNJ 0.0%
JPM 0.0%
K 0.0%
MA 17.02%
NFLX 17.27%
UL 0.0%
WBA 1.13%


In [34]:
# cleaned_weights_best_sharpe: orderedDict of ratios of each stock
# portfolio_dailychanges.columns: AAPL, CAH, etc
# print(portfolio_daily_changes)
# portfolio_daily_changes.drop(columns=['netDailyChange'], inplace=True)
# for i in portfolio_daily_changes.columns:
#     print(i, cleaned_weights_best_sharpe[i])

In [35]:
portfolio_daily_changes['netDailyChange'] = 0

# TODO: WHY is the for loop not working??
# for i in portfolio_daily_changes.columns:
#     portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe[i] * portfolio_daily_changes[i]
    
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['AAPL'] * portfolio_daily_changes['AAPL']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['CAH'] * portfolio_daily_changes['CAH']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['CMCSA'] * portfolio_daily_changes['CMCSA']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['DISH'] * portfolio_daily_changes['DISH']
# portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['FB'] * portfolio_daily_changes['FB']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['GOOG'] * portfolio_daily_changes['GOOG']
# portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['GS-PJ'] * portfolio_daily_changes['GS-PJ']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['HSY'] * portfolio_daily_changes['HSY']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['JNJ'] * portfolio_daily_changes['JNJ']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['JPM'] * portfolio_daily_changes['JPM']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['K'] * portfolio_daily_changes['K']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['MA'] * portfolio_daily_changes['MA']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['NFLX'] * portfolio_daily_changes['NFLX']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['UL'] * portfolio_daily_changes['UL']
portfolio_daily_changes['netDailyChange'] += cleaned_weights_best_sharpe['WBA'] * portfolio_daily_changes['WBA']
# portfolio_daily_changes
# portfolio_daily_changes.drop(columns=['netDailyChange'], inplace=True)

In [36]:
def total_return(data):
    initial = 1
    for index, row in data.items():
        # ?: ignore the first row because the portfolio starts at $100,000 on that day
        if index == portfolio_daily_changes.index[0]:
            continue

        row += 1
        initial *= row
    return initial

In [37]:
# TOTAL RETURNS AND SHARPE RATIO FOR TRAINING TIMEFRAME(2009-2016)
sharpe = sharpe_ratio(portfolio_daily_changes.loc['2009-01-01':'2017-01-01']['netDailyChange'])
totalReturns = total_return(portfolio_daily_changes.loc['2009-01-01':'2017-01-01']['netDailyChange'])
totalReturns, sharpe
# portfolio_daily_changes.loc['2009-01-01':'2017-01-01']['netDailyChange']

(11.62308611414271, 1.509233447254264)

In [54]:
# TOTAL RETURNS AND SHARPE RATIO FOR TESTING TIMEFRAME(2017-2018)
sharpe = sharpe_ratio(portfolio_daily_changes.loc['2017-01-01':'2019-01-01']['netDailyChange'])
totalReturns = total_return(portfolio_daily_changes.loc['2017-01-01':'2019-01-01']['netDailyChange'])
totalReturns, sharpe


portfolio_daily_changes.loc['2017-01-01':'2019-01-01']['netDailyChange']

Date
2017-01-03    0.011491
2017-01-04    0.004424
2017-01-05    0.005582
2017-01-06    0.007230
2017-01-09    0.002228
2017-01-10    0.000475
2017-01-11    0.004445
2017-01-12   -0.002576
2017-01-13    0.005743
2017-01-17    0.002421
2017-01-18    0.001501
2017-01-19    0.004943
2017-01-20    0.003347
2017-01-23   -0.001782
2017-01-24    0.006071
2017-01-25    0.006074
2017-01-26   -0.004017
2017-01-27    0.005159
2017-01-30   -0.004478
2017-01-31   -0.005490
2017-02-01    0.023198
2017-02-02   -0.001586
2017-02-03    0.008194
2017-02-06    0.005338
2017-02-07    0.007308
2017-02-08    0.002448
2017-02-09    0.005522
2017-02-10   -0.000869
2017-02-13    0.004841
2017-02-14    0.003817
2017-02-15    0.005705
2017-02-16   -0.003313
2017-02-17    0.000900
2017-02-21    0.007010
2017-02-22    0.001695
2017-02-23    0.000144
2017-02-24    0.002095
2017-02-27   -0.001744
2017-02-28   -0.000856
2017-03-01    0.014105
2017-03-02   -0.008624
2017-03-03    0.001692
2017-03-06    0.001518
2017-0

In [57]:
# TOTAL RETURNS AND SHARPE RATIO FOR ENTIRE TIMEFRAME(2009-2018)
sharpe = sharpe_ratio(portfolio_daily_changes['netDailyChange'])
totalReturns = total_return(portfolio_daily_changes['netDailyChange'])
totalReturns, sharpe
std_dev(portfolio_daily_changes['netDailyChange'])

0.012842560204285168

In [40]:
# min vol: (1.2510565075695712, 0.4156715453255877)
# best sharpe: (2.370754712244849, 1.1054096304284171)
mutated_data = portfolio_daily_changes.drop(columns=["AAPL", "CAH", "CMCSA", "DISH", "GOOG", "HSY", "JNJ", "JPM", "K", "MA", "NFLX", "UL", "WBA"])

# !: The $100,000 is put into the portfolio after day 0 close so it is not affected by price changes on day 0
mutated_data.loc[mutated_data.index[0], 'Portfolio Value'] = 100000
for i in range(1, len(mutated_data)):
    mutated_data.loc[mutated_data.index[i], 'Portfolio Value'] = mutated_data.loc[mutated_data.index[i-1], 'Portfolio Value'] * mutated_data.loc[mutated_data.index[i], 'netDailyChange'] + mutated_data.loc[mutated_data.index[i-1], 'Portfolio Value']

mutated_data.tail()

,netDailyChange,Portfolio Value
Date,,
2018-12-24,-0.026028,1.631467e+06
2018-12-26,0.061571,1.731918e+06
2018-12-27,0.001434,1.734401e+06
2018-12-28,-0.001122,1.732455e+06
2018-12-31,0.015768,1.759772e+06


In [41]:
mutated_data.to_csv('portfolioBestSharpe.csv', index=True)

### Sharpe and Total Returns for max sharpe portfolio
- Training Timeframe: (11.62274549145041, 1.5092151864491001)
- Testing Timeframe: (1.5140309299877148, 1.1038966069959586)
- Total Timeframe: (17.597196165431157, 1.4340746076171493)

### Sharpe and Total Returns for min volatility portfolio
- Training Timeframe: (2.9372727423958414, 0.9941734735774026)
- Testing Timeframe: (1.0944363130248105, 0.25940771449832484)
- Total Timeframe: (3.2146579505359787, 0.8440027044260568)

In [42]:
balance=0

def setDate(y, m, d):
    return y + "-" + m + "-" + d 

#Monthly change

'''
dayStart="2009-01-01"
dayEnd="2010-12-31"
i = 1
while(i != 25):
    dayStart_date = dayStart[8:10]
    dayStart_month = dayStart[5:7]
    dayStart_year = dayStart[0:4]
    if int(dayStart_month) == 12:
            dayStart_month = "02"
            dayStart_year = str(int(dayStart_year) + 1)
            if len(dayStart_year) == 1: dayStart_year = "0" + dayStart_year
    else:
        dayStart_month = str(int(dayStart_month) + 1)
        if len(dayStart_month) == 1: dayStart_month = "0" + dayStart_month
    dayStart = setDate(dayStart_year, dayStart_month, dayStart_date)
    print(dayStart)
    i=i+1
'''

y = 2009
m = 1
d = 1
while(y < 2011):
    dayStart = datetime.datetime(y, m, d)
    if (m < 12):
        m = m+1
        dayEnd = datetime.datetime(y, m, d)
    else:
        y = y+1
        m = 1
        dayEnd = datetime.datetime(y+1, m, d)
    print(get_stock("AAPL", dayStart, dayEnd).describe())


            AAPL
count  21.000000
mean    3.164711
std     0.154712
min     2.792857
25%     3.048214
50%     3.201429
75%     3.250357
max     3.377857
            AAPL
count  19.000000
mean    3.359793
std     0.153396
min     3.105357
25%     3.246428
50%     3.341071
75%     3.475893
max     3.661071
            AAPL
count  23.000000
mean    3.499689
std     0.299035
min     2.968214
25%     3.214464
50%     3.559286
75%     3.778750
max     3.923929
            AAPL
count  23.000000
mean    4.274534
std     0.198573
min     3.754286
25%     4.177857
50%     4.303571
75%     4.425000
max     4.544286
            AAPL
count  22.000000
mean    4.591169
std     0.179248
min     4.267500
25%     4.456161
50%     4.580536
75%     4.728304
max     4.976786
            AAPL
count  23.000000
mean    4.988276
std     0.111027
min     4.786071
25%     4.880715
50%     4.995000
75%     5.086964
max     5.166786
            AAPL
count  23.000000
mean    5.322096
std     0.341391
min     4.8357